Part 2: Custom

Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.neural_network
import sklearn.model_selection
import torch
from torchvision.transforms import v2
import torchvision.models as models
from PIL import Image
from tensorflow.keras.applications import MobileNetV3Large, mobilenet_v3


Load in data

In [2]:
data_dir = os.path.abspath("data_fashion/")

train_x = pd.read_csv(os.path.join(data_dir, "x_train.csv")).to_numpy()
train_y = pd.read_csv(os.path.join(data_dir, "y_train.csv")).to_numpy()[:,1].astype(np.int32)

valid_x = pd.read_csv(os.path.join(data_dir, "x_valid.csv")).to_numpy()
valid_y = pd.read_csv(os.path.join(data_dir, "y_valid.csv")).to_numpy()[:,1].astype(np.int32)

test_x = pd.read_csv(os.path.join(data_dir, "x_test.csv")).to_numpy()

print(test_x.shape)

(600, 784)


Function to Change to RGB tensor for pretrained model

In [3]:

def toImage(image):

    img = np.zeros((28, 28, 3), dtype=np.uint8)

    for i in range(28):
        row = image[(i*28):(i*28) + 28]

        for j in range(len(row)):
            img[i][j][0] = np.uint8(row[j])
            img[i][j][1] = np.uint8(row[j])
            img[i][j][2] = np.uint8(row[j])

    # img = np.zeros((28, 28), dtype=np.uint8)

    # for i in range(28):
    #     row = image[(i*28):(i*28) + 28]

    #     for j in range(len(row)):
    #         img[i][j] = np.uint8(row[j])

    pil_image = Image.fromarray(img)
    pil_image = pil_image.resize((224, 224))
    # tensor = torch.tensor(img.transpose((2,0,1)), dtype=torch.uint8)

    array = np.array(pil_image)

    return array


In [ ]:
print(toImage(train_x[0]))

Change to Correct format (PIL images)

In [4]:

train_x = np.apply_along_axis(toImage, axis=1, arr=train_x)
test_x = np.apply_along_axis(toImage, axis=1, arr=test_x)
valid_x = np.apply_along_axis(toImage, axis=1, arr=valid_x)



In [6]:
print(train_x.shape)

(2102, 224, 224, 3)


In [34]:
print(type(train_x))
print(type(test_x))
print(type(valid_x))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


Import the pretrained model and apply transformations that go along with corresponding weights

In [7]:
def callModelOn(batch):
    pretrained_model = MobileNetV3Large()
 
    batch = mobilenet_v3.preprocess_input(batch)

    # call prediction
    prediction = pretrained_model(batch)

    return prediction



In [19]:
# def callModelOn(array):
#     batch = torch.tensor(array)

#     # Get the model and corresponding weights
#     weights = models.MobileNet_V3_Large_Weights.IMAGENET1K_V2
#     pretrained_model = models.mobilenet_v3_large(weights=weights)
#     pretrained_model.eval()

#     # initialize transformation - transforms images to correct size
#     preprocess = weights.transforms()

#     # apply the transforms
#     batch = torch.tensor([preprocess(tensor) for tensor in batch])

#     # call prediction
#     prediction = pretrained_model(batch)

#     return prediction

# batch = torch.tensor(toImage(train_x[0]))

# # # Get the model and corresponding weights
# weights = models.MobileNet_V3_Large_Weights.IMAGENET1K_V2
# pretrained_model = models.get_model("mobilenet_v3_large", weights=weights)
# pretrained_model.eval()

# # initialize transformation - transforms images to correct size
# preprocess = weights.transforms()

# # apply the transforms
# batch = batch.apply(preprocess)

# # call prediction
# prediction = pretrained_model(batch)



In [8]:
mobile_train = np.array(callModelOn(train_x))
mobile_valid = np.array(callModelOn(valid_x))

In [9]:
base_mlp = sklearn.neural_network.MLPClassifier(
    hidden_layer_sizes=(64,),
    activation='relu',
    solver='lbfgs',
)


In [14]:
# custom splitter code

xall_LF = np.vstack([mobile_train, mobile_valid])
yall_L = np.hstack([train_y, valid_y])

valid_indicators_L = np.hstack([
    -1 * np.ones(train_y.size),
    0  * np.ones(valid_y.size),
    ])

my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_L)

In [15]:
mlp_hyperparam_grid = dict(
    hidden_layer_sizes=[(32,), (64,), (128,)],
    alpha=np.logspace(-4, 4, 9),
    random_state=[i for i in range(4)],
)

In [16]:
mlp_grid_searcher = sklearn.model_selection.GridSearchCV(
    base_mlp, mlp_hyperparam_grid, scoring='balanced_accuracy', 
    cv=my_splitter, return_train_score=True, refit=False)

In [17]:
mlp_grid_searcher.fit(xall_LF, yall_L)

c:\Users\gmcgu\micromamba\envs\cs135_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\gmcgu\micromamba\envs\cs135_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\gmcgu\micromamba\envs\cs135_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs faile

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=MLPClassifier(hidden_layer_sizes=(64,), solver='lbfgs'),
             param_grid={'alpha': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03,
       1.e+04]),
                         'hidden_layer_sizes': [(32,), (64,), (128,)],
                         'random_state': [0, 1, 2, 3]},
             refit=False, return_train_score=True, scoring='balanced_accuracy')

In [18]:
best_mlp = base_mlp.set_params(**mlp_grid_searcher.best_params_)
best_mlp.fit(xall_LF, yall_L)

c:\Users\gmcgu\micromamba\envs\cs135_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=0.01, hidden_layer_sizes=(64,), random_state=3,
              solver='lbfgs')

In [19]:
yhat_valid = base_mlp.predict(mobile_valid)
sklearn.metrics.balanced_accuracy_score(valid_y, yhat_valid)

0.9033333333333333

In [20]:
# mapping of classification id to object name
classifications = {
    3 : "dress",
    2 : "pullover",
    0 : "top",
    1 : "trouser",
    5 : "sandal",
    7 : "sneaker",
}

In [21]:
mobile_test = np.array(callModelOn(test_x))

In [ ]:
yhat_test = best_mlp.predict(mobile_test)

In [ ]:
predictions = [classifications[yhat] for yhat in yhat_test]
np.savetxt('yhat_test.txt', predictions, delimiter='\n', fmt='%s')